### Sales agent sending to multiple recepients

In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio



In [2]:
load_dotenv(override=True)

True

In [3]:

subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")


In [4]:
@function_tool
def send_html_email(to_email:str, subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("ainanihsan@gmail.com")  # Change to your verified sender
    to_email = To(to_email)  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [5]:
tools = [subject_tool, html_tool, send_html_email]

In [6]:
tools

[FunctionTool(name='subject_writer', description='Write a subject for a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000023FE5971620>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='html_converter', description='Convert a text email body to an HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000023FE5971760>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='send_html_email', description='Send out an email with the given subject and HTML body to 

In [8]:
instructions = """
You are an Email Manager at ComplAI.

Your responsibilities:
1. Receive a single email template (body) with placeholders like {first_name}, {company}, etc.
2. Receive a list of recipients in the input, under the key 'recipients'. Each recipient is a dictionary with keys matching the placeholders (e.g., first_name, company, email).
3. For each recipient:
   a. Fill in the placeholders in the body.
   b. Use the subject_writer tool to generate a subject for the email.
   c. Use the html_converter tool to convert the body to HTML.
   d. Use the send_html_email tool to send the email with the personalized subject and HTML body, passing the recipient's email as the 'to_email' argument.
4. Return a summary with:
   - Number of emails successfully sent
   - Number of failures, if any
"""

emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=[subject_tool, html_tool, send_html_email],  # only the tools needed
    model="gpt-4o-mini",
    handoff_description="Send personalized emails to multiple recipients"
)


In [10]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [11]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini",
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini",
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini",
)

In [12]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

### Now we have 3 tools and 1 handoff

In [13]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000023FE5B5E840>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000023FE5B5F060>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}

In [ ]:
# --- UPDATED SALES MANAGER INSTRUCTIONS ---
sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.

Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts.
   - Each draft should be a **template** using placeholders like {first_name} and {company} instead of real names or emails.
   - Do not include actual recipient names or addresses in the draft.
2. Evaluate and Select: Review the drafts and choose the single best email template using your judgment of which one is most effective.
   - You can use the tools multiple times if you're not satisfied with the results from the first try.
3. Handoff for Sending: Pass ONLY the winning email template to the 'Email Manager' agent.
   - Include the recipients list in the string you provide so the Email Manager can fill in the placeholders and send personalized emails.
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email template to the Email Manager — never more than one.
"""

sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,         # list of sales agent tools
    handoffs=handoffs,   # list containing emailer_agent
    model="gpt-4o-mini"
)
# Your original recipients list
recipients = [
    {"first_name": "z", "company": "KTH", "email": "z@gmail.com"},
    {"first_name": "a", "company": "Upwork", "email": "a@gmail.com"},
    {"first_name": "m", "company": "RYK", "email": "m@gmail.com"},

]

# Convert it to text that the agent can understand
recipient_text = "\n".join(
    [f"{r['first_name']} at {r['company']} <{r['email']}>" for r in recipients]
)

# Combine the task and recipients into a single string
message_text = f"""
Task: Send out a cold sales email addressed to Dear CEO from Alice
Recipients:
{recipient_text}
"""

# --- RUN SALES MANAGER ---
with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message_text)


### Remember to check the trace

https://platform.openai.com/traces

And then check your email!!